In [26]:
#..........................案例1:
#.......................... 理解朴素贝叶斯
# 之所以叫朴素贝叶斯，是因为假设很原始
# 假设各个特征相互独立；各个特征同等重要。（虽然事实并非如此，但是效果不错）
# 贝叶斯的精髓在于求p(A|B)变成求p(B|A)
postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
classVec = [0,1,0,1,0,1]    #1表示侮辱性的，0表示非侮辱性的
# 收集所有词构造unique列表（假设：相互独立）
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)
# 查看inputSet的每个词是否在vocabList中
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1 # 在的改为1，不在保持为0
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec
# 修正setOfWords2Vec，以下代码并没有用修正的模型，以后可以考虑用这个
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1 # 可能某些词多次出现
    return returnVec
print("两个函数的作用结果",setOfWords2Vec(createVocabList(postingList),postingList[0]))
# 寻找侮辱性词汇
# 原理：侮辱性语句中，出现概率高的词很有可能具有侮辱的代表性
# 贝叶斯：给定侮辱性/非侮辱性的文档，在文档中找词出现的可能性（条件概率）
import numpy as np
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs) # 侮辱性的概率
    # 后续计算p(w|c)=p(w1|c)...p(wn|c)时，避免0概率的影响，取np.ones()，w代表侮辱性句子，wi代表侮辱性词汇
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)
    p0Denom = 2.0; p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i] # 词典中，单个侮辱性词出现的个数
            p1Denom += sum(trainMatrix[i]) # 词典中，侮辱性语句所有词出现的总数
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)  # 取log，概率不至于太小
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive
Voclist=createVocabList(postingList)
trainMat=[]
# 看哪些字出现频率较高会被定义为侮辱性的或者非侮辱性的
for i in postingList:
    trainMat.append(setOfWords2Vec(Voclist,i))
p0V,p1V,pAB=trainNB0(trainMat,classVec)
print(p0V,p1V,pAB)
# p1V 给定文档类别（侮辱性）下词汇表中侮辱性/非侮辱性单词出现的概率。概率越大，（且显著大于出现在非侮辱性语句中的概率），越可能代表侮辱性

# 测试
# 给定一个句子，判断每个词全在侮辱性环境里的概率
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
testEntry = ['love', 'my', 'dalmation']
# Voclist 训练的结果
# p0V,p1V,pAB 训练结果的p值
thisDoc = np.array(setOfWords2Vec(Voclist, testEntry))
print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAB))

两个函数的作用结果 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[-2.56494936 -2.56494936 -3.25809654 -3.25809654 -2.56494936 -1.87180218
 -2.56494936 -2.56494936 -3.25809654 -3.25809654 -2.56494936 -2.56494936
 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -3.25809654
 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -2.15948425 -2.56494936 -3.25809654 -3.25809654 -3.25809654
 -3.25809654 -2.56494936] [-3.04452244 -3.04452244 -2.35137526 -2.35137526 -2.35137526 -3.04452244
 -3.04452244 -3.04452244 -2.35137526 -2.35137526 -3.04452244 -3.04452244
 -3.04452244 -3.04452244 -2.35137526 -1.94591015 -3.04452244 -1.94591015
 -3.04452244 -1.65822808 -3.04452244 -3.04452244 -2.35137526 -3.04452244
 -3.04452244 -2.35137526 -3.04452244 -2.35137526 -2.35137526 -2.35137526
 -2.35137526 -3.04452244] 0.5
['love', 'my', 'dalmation'] classified as:  0


In [1]:
#..........................案例1:
#.......................... 贝叶斯应用
#..........................处理文档的可能性不大，有空再展开
